<center> <h1> Lecture 6: Embeddings and Model Selection </h1> </center>
<center> Jillian Fisher, Zaid Harchaoui </center>
    <center> Data 598 (Winter 2023), University of Washington </center>

We will discuss two topics this lecture:
- Embeddings for natural language
- Model Selection with statistical tests


The first part of this notebook has been adapted from the [D2L book]( http://d2l.ai/chapter_natural-language-processing-pretraining/similarity-analogy.html),  and adapted from lecture material created by Krishna Pillutla.

# Part 1: Embeddings for Natural Language

The field of **natural language processing (NLP)** is concerned with the interaction between computers and natural (human) language. This involves "understanding" the contents of documents, including the contextual nuances of the language within them. 

**Embeddings**:
The use of machine learning for NLP, both in the classical settings as well as the modern deep learning era, have relied on *embedding* words in vector spaces.
Words are made of characters, which are combinatorial in nature with no "neighborhood" structure which one expects of vectors in, say, a Euclidean space. 
The magic of embeddings is that they are able to capture some "neighborhood" structure in words, e.g., the embedding of semantically similar words are closer together than of words which have nothing in common. 

![](https://miro.medium.com/max/2400/1*OEmWDt4eztOcm5pr2QbxfA.png)
Image credits: https://towardsdatascience.com/creating-word-embeddings-coding-the-word2vec-algorithm-in-python-using-deep-learning-b337d0ba17a8

**Note**: Sometimes, we will work at the level of subword units, rather than words. Mathematically, the same treatment holds irrespective of how we *tokenize* the text. We will refer to these units as *tokens*.


**Types of embeddings**:

- Global embeddings: word2vec, GloVe
- Contextual embeddings: ELMo, BERT, ...

In this lab, we will play with the GloVe embeddings, which are global embeddings. 


In [ ]:
import numpy as np

In [ ]:
# Download the GloVe embedding ~66M compressed + 164M uncompressed
import os
if 'glove.6B.50d' not in os.listdir():
    !wget http://d2l-data.s3-accelerate.amazonaws.com/glove.6B.50d.zip
    !unzip glove.6B.50d.zip

We index each word in our dictionary using integers. 
We store the following mappings:
- word $\to$ index
- index $\to$ word
- index $\to$ embedding of the corresponding word

Words not in our dictionary are denoted using the `<unk>` token

In [ ]:
class GloVeWordEmbedding:
    def __init__(self):
        self.idx_to_token, self.idx_to_vec = self._load_embedding()
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self):
        idx_to_token, idx_to_vec = ['<unk>'], []
        with open('glove.6B.50d/vec.txt') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, np.asarray(idx_to_vec)

    def __getitem__(self, tokens):
        # "tokens" is a list of words
        # use as object[tokens]
        # map token -> index -> vector
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[np.asarray(indices)]
        return vecs
    
    def __call__(self, tokens):
        # Use as object(tokens)
        return self.__getitem__(tokens)
    
    def __len__(self):
        return len(self.idx_to_token)

In [ ]:
glove_embedding = GloVeWordEmbedding()

We start by noting that the embedding of a word does not depend on its context. Recall that GloVe is a global (non-contextual) embedding in contrast to contextual embeddings such as BERT.

In [ ]:
# To obtain the embeddings of words:
sentence1 = 'I love data science'
embeddings1 = glove_embedding[sentence1.split()]  # using the __getitem__ method
# alternatively: 
# embeddings1 = glove_embedding(sentence1.split())  # using the __call__ method
print(embeddings1.shape)  # (number of words, dimension)

(4, 50)


In [ ]:
sentence2 = 'As a kid, I always wanted to study mathematics and science'
embeddings2 = glove_embedding[sentence2.split()]
print(embeddings2.shape)  # (number of words, dimension)

(11, 50)


In [ ]:
# Compare the embeddings of both the time the word "science" appears
e1 = embeddings1[-1]
e2 = embeddings2[-1]
print(np.linalg.norm(e1-e2))

0.0


Next we will look at the cosine similarity between word embeddings. 
Recall that the cosine similarity between two vectors $u, v \in \mathbb{R}^d$ is defined as 
$$
    S_{\cos}(u, v) = \frac{\langle u, v\rangle}{\|u\|_2  \, \|v\|_2} .
$$

For any pair of vectors, the cosine similarity is always 
between $-1$ and $1$. (Why?)

In [ ]:
# Write a function to compute the k nearest neighbors 
# as per cosine similarity
def k_nearest_neighbors(population, query, k):
    # population is a matrix of size (n, dim)
    # query is a matrix of shape (1, dim)
    # k is an integer
    # return topk indices and topk values
    cos = np.dot(population, query.reshape(-1,)) / (
        np.sqrt(np.sum(population * population, axis=1) + 1e-9) *
        np.linalg.norm(query.reshape(-1))
    )
    topk_idx = np.argpartition(cos, -k)[-k:] # unsorted
    topk_idx = topk_idx[np.argsort(cos[topk_idx])][::-1]  # sorted
    topk_val = cos[topk_idx]
    return topk_idx, topk_val

# Hint for topk in numpy: https://stackoverflow.com/a/23734295

In [ ]:
query = glove_embedding(['microwave'])
topk_idxs, topk_vals = k_nearest_neighbors(glove_embedding.idx_to_vec, query, 6)
topk_words = [glove_embedding.idx_to_token[i] for i in topk_idxs]

for i, (w, val) in enumerate(zip(topk_words, topk_vals)):
    print(i, w, val)

0 microwave 0.9999999999839023
1 analog 0.7300107691175304
2 microwaves 0.7264979322621885
3 oven 0.7115686481570183
4 refrigerator 0.7039825402692076
5 ovens 0.6948281242683833


In [ ]:
query = glove_embedding(['serendipity'])
topk_idxs, topk_vals = k_nearest_neighbors(glove_embedding.idx_to_vec, query, 6)
topk_words = [glove_embedding.idx_to_token[i] for i in topk_idxs]

for i, (w, val) in enumerate(zip(topk_words, topk_vals)):
    print(i, w, val)

0 serendipity 0.9999999999485274
1 profundity 0.7222786942557122
2 happenstance 0.7016944428733708
3 strangeness 0.6975966805924638
4 weirdness 0.6901399894427984
5 hokum 0.6882356254537078


### Analogies with word embeddings

In addition to seeking synonyms, we can also use the pretrained word vector to seek the analogies between words. For example, “man”:“woman”::“son”:“daughter” is an example of analogy, “man” is to “woman” as “son” is to “daughter”. 

The problem of seeking analogies can be defined as follows: for four words in the analogical relationship $a:b::c:d$, given the first three words, a, b and c, we want to find d. 

Assume the word vector for the word w is $\text{vec}(w)$. To solve the analogy problem, we need to find the word vector that is most similar to the result vector of $\text{vec}(c)+\text{vec}(b)−\text{vec}(a)$.

In [ ]:
def get_analogy(token_a, token_b, token_c):
    # Implement the analogy from above
    vecs = glove_embedding[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = k_nearest_neighbors(glove_embedding.idx_to_vec, x, 1)
    return glove_embedding.idx_to_token[int(topk[0])]  # Remove unknown words

In [ ]:
get_analogy('man', 'woman', 'son')

'daughter'

In [ ]:
# capital-country
get_analogy('london', 'england', 'paris')

'france'

In [ ]:
# tense
get_analogy('dance', 'danced', 'look')

'looked'

# Part 2: Model Selection with Statistical Tests

We will run a test to compare two different models. This is called the McNemar's test. 

Let $h_1$ and $h_2$ be two different classification algorithms. 
The hypotheses we're testing are:
$$
H_0 : \quad \text{acc}(h_1) = \text{acc}(h_2) \\
H_1 : \quad \text{acc}(h_1) \ne \text{acc}(h_2) ,
$$
where acc $(h)$ is the accuracy of the classifier $h$.


To distinguish between the two, we compute the following numbers:
- $N_{01}$: the number of validation examples misclassified by $h_1$ but correctly classified by $h_2$
- $N_{10}$: the number of validation examples correctly classified by $h_1$ but misclassified by $h_2$. 

The test statistic is 
$$
    T = \frac{(|N_{01} - N_{10}| - 1)^2}{N_{10} + N_{01}}.
$$
Its asymptotic distribution under the null is $\chi^2$-distribution with $1$ degree of freedom. We reject the test null hypothesis if 
$$T > \chi^2_{1, \alpha}$$
the $(1-\alpha)$-quantile of the $\chi^2_1$ distribution.

**The exercise**:
Run this hypothesis test to compare the MLP from week 1 and the ConvNet from week 2. Use a significance $\alpha=0.01$. Train each network with SGD for $30$ passes with an appropriate learning rate.

### Data 

In [ ]:
import numpy as np
import torch
from torchvision.datasets import MNIST, FashionMNIST
from torch.nn.functional import cross_entropy
import time
import scipy.stats


In [ ]:
# download dataset (~117M in size)
train_dataset = FashionMNIST('./data', train=True, download=True)
X_train = train_dataset.data # torch tensor of type uint8
y_train = train_dataset.targets # torch tensor of type Long
test_dataset = FashionMNIST('./data', train=False, download=True)
X_test = test_dataset.data
y_test = test_dataset.targets

# choose a subsample of 10% of the data:
idxs_train = torch.from_numpy(
    np.random.choice(X_train.shape[0], replace=False, size=X_train.shape[0]//10))
X_train, y_train = X_train[idxs_train], y_train[idxs_train]
# idxs_test = torch.from_numpy(
#     np.random.choice(X_test.shape[0], replace=False, size=X_test.shape[0]//10))
# X_test, y_test = X_test[idxs_test], y_test[idxs_test]

print(f'X_train.shape = {X_train.shape}')
print(f'n_train: {X_train.shape[0]}, n_test: {X_test.shape[0]}')
print(f'Image size: {X_train.shape[1:]}')

# Normalize dataset: pixel values lie between 0 and 255
# Normalize them so the pixelwise mean is zero and standard deviation is 1

X_train = X_train.float()  # convert to float32
X_train = X_train.view(-1, 784)
mean, std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean[None, :]) / (std[None, :] + 1e-6)  # avoid divide by zero

X_test = X_test.float()
X_test = X_test.view(-1, 784)
X_test = (X_test - mean[None, :]) / (std[None, :] + 1e-6)

n_class = np.unique(y_train).shape[0]

X_train.shape = torch.Size([6000, 28, 28])
n_train: 6000, n_test: 10000
Image size: torch.Size([28, 28])


### Modules and SGD

In [ ]:
import torch
class MLP(torch.nn.Module): 
    def __init__(self, hidden_width=32):
        super().__init__()
        self.linear1 = torch.nn.Linear(784, hidden_width)
        self.linear2 = torch.nn.Linear(32, 10)
    def forward(self, x):
        x = self.linear1(x)
        x = torch.nn.functional.relu(x)
        x = self.linear2(x)
        return x

class ConvNet(torch.nn.Module):
    def __init__(self,num_classes=10):
        super().__init__()
        self.conv_ensemble_1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        self.conv_ensemble_2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        self.fc = torch.nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        out = self.conv_ensemble_1(x)
        out = self.conv_ensemble_2(out)
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        return out
    
# Some utility functions to compute the objective and the accuracy
def compute_objective(model, X, y):
    score = model(X)
    # PyTorch's function cross_entropy computes the multinomial logistic loss
    return cross_entropy(input=score, target=y, reduction='mean') 

def sgd_one_pass(model, X, y, learning_rate, verbose=False):
    num_examples = X.shape[0]
    average_loss = 0.0
    for i in range(num_examples):
        idx = np.random.choice(X.shape[0])
        # compute the objective. 
        # Note: This function requires X to be of shape (n,d). In this case, n=1 
        objective = compute_objective(model, X[idx:idx+1], y[idx:idx+1]) 
        average_loss = 0.99 * average_loss + 0.01 * objective.item()
        if verbose and (i+1) % 100 == 0:
            print(average_loss)
        
        # compute the gradient using automatic differentiation
        gradients = torch.autograd.grad(outputs=objective, inputs=model.parameters())
        
        # perform SGD update. IMPORTANT: Make the update inplace!
        for (w, g) in zip(model.parameters(), gradients):
            w.data -= learning_rate * g.data
      
    
from tqdm.auto import trange # range + progress bar
def sgd_n_passes(model, X_train, y_train, X_val, y_val, n_passes, learning_rate):
    for i in trange(n_passes):
        sgd_one_pass(model, X_train, y_train, learning_rate)
    return compute_prediction_performance(model, X_val, y_val)

@torch.no_grad()
def compute_prediction_performance(model, X, y):
    # return a boolean vector of the same length as y
    # each entry is True if correctly predicted, else False
    score = model(X)
    predictions = torch.argmax(score, axis=1)  # class with highest score is predicted
    return (predictions == y)  

In [ ]:
model1 = MLP()
performance1 = sgd_n_passes(
    model1, X_train, y_train, X_test, y_test, n_passes=30, learning_rate=2e-3
)
# boolean vector of length n_test

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
model2 = ConvNet()
performance2 = sgd_n_passes(
    model2, X_train, y_train, X_test, y_test, n_passes=30, learning_rate=2.5e-3
)
# boolean vector of length n_test

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
print('accuracy of MLP:', performance1.sum().item()/y_test.shape[0])
print('accuracy of ConvNet:', performance2.sum().item()/y_test.shape[0])

accuracy of MLP: 0.8324
accuracy of ConvNet: 0.8698


We will test whether this difference is statistically significant or not. From a 
first glance, it does appear to be statistically significant as the gap is quite large. 

Compute $N_{01}$ and $N_{10}$ from the output of SGD above.

In [ ]:
N01 = (~performance1 & performance2).sum().item()  # MLP is wrong but ConvNet is correct
N10 = (performance1 & ~performance2).sum().item()  # MLP is correct but ConvNet is wrong

Now compute the test statistic, the threshold, which is the $(1-\alpha)$ quantile of the $\chi^2_1$ distribution and read off the conclusion of the test. Recall that we have $\alpha=0.01$ here.

In [ ]:
T = (abs(N01 - N10) - 1)**2 / (N01 + N10)  # statistic
threshold = scipy.stats.chi2(df=1).ppf(0.99)

print(f'Test statistic: {T}, threshold: {threshold}')

if T > threshold:
    print('Null rejected')
else:
    print('Failed to reject the null')

Test statistic: 128.58502772643254, threshold: 6.6348966010212145
Null rejected
